In [ ]:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Licencinho/PMA/blob/main/run_PMA_flask.ipynb)

In [2]:
!pip install flask-ngrok
!pip install pyngrok
!pip install flask-wtf

In [3]:
!ngrok authtoken "26BvbMOktqjiQ2Ibfb05r7jzTcL_4UJaagneSjXiB6qeUZXzT"

Authtoken saved to configuration file: C:\Users\alons/.ngrok2/ngrok.yml


In [5]:
!git clone "https://github.com/Licencinho/PMA"
import os
os.chdir("/content/PMA")

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '/content/PMA'

In [10]:
from flask import *
from flask_ngrok import run_with_ngrok
import pandas as pd
from flask_wtf import FlaskForm
from wtforms import StringField, DateTimeField, SubmitField, EmailField
from wtforms.validators import DataRequired
from wtforms import validators

app = Flask(__name__)
app.config['SECRET_KEY'] = 'super-secret-impossible-impopossible' # don't do this in production

from flask_wtf.csrf import CSRFProtect
csrf = CSRFProtect(app)

app_data = {
    "name":         "Dissertation Planner",
    "description":  "A basic Flask app for students to plan their dissertations",
    "author":       "Peter Simeth",
    "html_title":   "Dissertation Planner",
    "project_name": "Dissertation Planner",
    "keywords":     "flask, webapp, dissertation, planning, python"
}

class UserForm(FlaskForm):
	name = StringField('Name', validators=[DataRequired()])
	email = EmailField('Email', validators=[DataRequired()])
	dob = DateTimeField('Date of Birth', format='%d/%m/%Y', validators=[DataRequired()])
	specialism = StringField('Specialism', validators=[DataRequired()])

@app.route('/')
def index():
    return render_template('index.html', app_data=app_data)


@app.route('/calendar')
def calendar():
    df = pd.read_csv("NamesTwo.csv")
    return render_template('calendar.html', df=df, app_data=app_data)


@app.route('/workload')
def workload():
    df = pd.read_csv("NamesTwo.csv")
    form = UserForm()
    return render_template('workload.html', df=df, form = form, app_data=app_data)


@app.route('/submit', methods=('GET', 'POST'))
def submit():
	form = UserForm()
	df = pd.read_csv("NamesTwo.csv")
	if request.method == "POST":
		if form.validate_on_submit():
			max_id = df['ID'].max()
			new_id = max_id + 1
			df2 = pd.DataFrame({'ID':[new_id], 
				'Name':[request.form.get("name")],
				'Email':[request.form.get("email")],
				'Age':[request.form.get("dob")],
				'Specialism':[request.form.get("specialism")]
			})
			df2['Age'] = pd.to_datetime(df2['Age'], dayfirst=True)
			now = pd.Timestamp('now')
			df2['Age'] = (now - df2['Age']).astype('<m8[Y]')
			df = pd.concat([df, df2], ignore_index=True)
			df.to_csv("NamesTwo.csv", index=False)

		return render_template('calendar.html', df=df, app_data=app_data)

	return render_template('workload.html', df=df, form=form, app_data=app_data)
	

run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-38:
Traceback (most recent call last):
  File "C:\Users\alons\anaconda3\lib\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\alons\anaconda3\lib\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "C:\Users\alons\anaconda3\lib\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\alons\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\alons\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 394, in 